# Conexión a la BBDD

In [ ]:
!pip install python-dotenv

In [1]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 18.3 MB/s eta 0:00:00


In [13]:
import mysql.connector
from dotenv import load_dotenv
import os

In [21]:
# cargar variables de entorno desde el archivo .env
load_dotenv()

# Configuración de la conexión
db_connection = mysql.connector.connect(
    host=os.getenv("DB_HOST"), # Host de la conexión CleverCloud-MySQLWorkbench
    user=os.getenv("DB_USER"), # Usuario de Clever Cloud
    password=os.getenv("DB_PASSWORD"), # Contraseña de Clever Cloud
    database=os.getenv("DB_NAME"), # Nombre de la base de datos de MySQLWorkbench
    port=3306, # Puerto de la conexión CleverCloud-MySQLWorkbench
    connection_timeout=600 # 10 min de tiempo de espera
)

cursor = db_connection.cursor()
print("Conexón exitosa a la base de datos de Clever Cloud")

Conexón exitosa a la base de datos de Clever Cloud


# Creación de la tabla en MySQL

In [17]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS productos_hombre (
        id INT AUTO_INCREMENT PRIMARY KEY,
        nombre VARCHAR(255),
        precio DECIMAL(10, 2),
        enlace_producto VARCHAR(255)
);
"""

cursor.execute(create_table_query)
db_connection.commit()
print("Tabla creada exitosamente")

Tabla creada exitosamente


# Web Scraping para la obtención de datos

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
# SCRAPEO QUE TANN SÓLO DEVUELVE EL PRIMER ARTÍCULO

# URL de la página web
url = "https://wodbox.es/es/43-hombre"

# Cabecera con un User-Agent que simula un navegador
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
}

# Solicitud a la página web
response = requests.get(url, headers=headers)

# Comprobar que la solicitud fue exitosa
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")

    print("La solicitud fue exitosa")

    # Extraer los productos
    productos = soup.find_all("div", class_="products")
    #print(productos)
    for producto in productos:
      nombre = producto.find("h3", class_="s_title_block").get_text()
      print(f"El nombre del producto es {nombre}")

      precio = producto.find("span", class_="price").get_text()
      print(f"El precio del producto es {precio}")

      # Obtener el enlace desde la etiqueta 'a' dentro del 'h3'
      enlace = producto.find("h3", class_="s_title_block").find("a")["href"]
      print(f"El enlace del producto es {enlace}")
else:
  print(f"Error en la solicitud: {response.status_code}")

La solicitud fue exitosa
El nombre del producto es Zapatillas de entrenamiento UA Reign Lifter unisex
El precio del producto es 200,00 €
El enlace del producto es https://wodbox.es/es/under-armour/7222-30432-zapatillas-de-entrenamiento-ua-reign-lifter-unisex.html#/41-tallaje_calzado-425


In [6]:
# SCRAPEO QUE DEVUELVE TODOS LOS ARTÍCULOS DE LA PÁGINA

import requests
from bs4 import BeautifulSoup

# URL de la página web
url = "https://wodbox.es/es/43-hombre"

# Cabecera con un User-Agent que simula un navegador
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
}

# Solicitud a la página web
response = requests.get(url, headers=headers)

# Comprobar que la solicitud fue exitosa
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")

    print("La solicitud fue exitosa")

    # Extraer los productos individuales utilizando el 'article' con clase 'ajax_block_product'
    productos = soup.find_all("article", class_="ajax_block_product")

    if not productos:
        print("No se encontraron productos")

    # Procesar cada producto
    for producto in productos:
        nombre = producto.find("h3", class_="s_title_block").get_text()
        print(f"El nombre del producto es {nombre}")

        precio = producto.find("span", class_="price").get_text()
        print(f"El precio del producto es {precio}")

        # Obtener el enlace desde la etiqueta 'a' dentro del 'h3'
        enlace = producto.find("h3", class_="s_title_block").find("a")["href"]
        print(f"El enlace del producto es {enlace}")
else:
    print(f"Error en la solicitud: {response.status_code}")

La solicitud fue exitosa
El nombre del producto es Zapatillas de entrenamiento UA Reign Lifter unisex
El precio del producto es 200,00 €
El enlace del producto es https://wodbox.es/es/under-armour/7222-30432-zapatillas-de-entrenamiento-ua-reign-lifter-unisex.html#/41-tallaje_calzado-425
El nombre del producto es Zapatillas de entrenamiento UA Reign Lifter unisex
El precio del producto es 200,00 €
El enlace del producto es https://wodbox.es/es/under-armour/7221-30435-zapatillas-de-entrenamiento-ua-reign-lifter-unisex.html#/23-tallaje_calzado-40
El nombre del producto es UNDER ARMOUR REIGN 6 TRAINING SHOES
El precio del producto es 119,00 €
El enlace del producto es https://wodbox.es/es/under-armour/7220-30430-under-armour-reign-6-training-shoes.html#/40-tallaje_calzado-42
El nombre del producto es UNDER ARMOUR REIGN 6 TRAINING SHOES
El precio del producto es 119,00 €
El enlace del producto es https://wodbox.es/es/under-armour/7219-30418-under-armour-reign-6-training-shoes.html#/37-talla

In [8]:
# SCRAPEO QUE DEVUELVE TODOS LOS ARTÍCULOS DE TODAS LAS PÁGINAS

import requests
from bs4 import BeautifulSoup
import mysql.connector

# Inicializar la página
pagina = 1
while True:
  url = f"https://wodbox.es/es/43-hombre?page={pagina}"
  print(f"Extrayendo productos de la página {pagina}...")

  # Cabecera con un User-Agent que simula un navegador
  headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
  }

  # Solicitud a la página web
  response = requests.get(url, headers=headers)

  # Comprobar que la solicitud fue exitosa
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    print(f"La solicitud fue exitosa para la página {pagina}")

    # Extraer los productos individuales utilizando el 'article' con clase 'ajax_block_product'
    productos = soup.find_all("article", class_="ajax_block_product")

    if not productos:
        print(f"No se encontraron productos en la página {pagina}")
        break

    # Procesar cada producto
    for producto in productos:
        nombre = producto.find("h3", class_="s_title_block").get_text().strip()
        precio = producto.find("span", class_="price").get_text().strip()
        enlace = producto.find("h3", class_="s_title_block").find("a")["href"]
        print(f"Producto: {nombre}, Precio: {precio}, Enlace: {enlace}")


    # Buscar el enlace a la siguiente página
    next_page = soup.find("a", class_="page-link next js-search-link")

    # Si no se encuentra la siguiente página, detener el bucle
    if not next_page:
      print("No se encontraron más páginas. Finalizando extracción")
      break

    # Incrementar el número de página
    pagina += 1
  else:
    print(f"Error en la solicitud para la página {pagina}: {response.status_code}")
    break

Extrayendo productos de la página 1...
La solicitud fue exitosa para la página 1
Producto: Zapatillas de entrenamiento UA Reign Lifter unisex, Precio: 200,00 €, Enlace: https://wodbox.es/es/under-armour/7222-30432-zapatillas-de-entrenamiento-ua-reign-lifter-unisex.html#/41-tallaje_calzado-425
Producto: Zapatillas de entrenamiento UA Reign Lifter unisex, Precio: 200,00 €, Enlace: https://wodbox.es/es/under-armour/7221-30435-zapatillas-de-entrenamiento-ua-reign-lifter-unisex.html#/23-tallaje_calzado-40
Producto: UNDER ARMOUR REIGN 6 TRAINING SHOES, Precio: 119,00 €, Enlace: https://wodbox.es/es/under-armour/7220-30430-under-armour-reign-6-training-shoes.html#/40-tallaje_calzado-42
Producto: UNDER ARMOUR REIGN 6 TRAINING SHOES, Precio: 119,00 €, Enlace: https://wodbox.es/es/under-armour/7219-30418-under-armour-reign-6-training-shoes.html#/37-tallaje_calzado-41
Producto: REEBOK NANO X3, Precio: 89,60 €, Enlace: https://wodbox.es/es/reebok/7204-30389-reebok-nano-x3.html#/41-tallaje_calzado-

# Guardar los datos en la base de datos

In [18]:
# Función para insertar productos en la base de datos
def insertar_producto(nombre, precio, enlace):
  insert_query = """
    INSERT INTO productos_hombre
    (nombre, precio, enlace_producto) VALUES (%s, %s, %s)
  """
  cursor.execute(insert_query, (nombre, precio, enlace))
  db_connection.commit()
  print("Datos insertados exitosamente")

In [22]:
# Importar los datos de todas las páginas
import requests
from bs4 import BeautifulSoup

# Inicializar la página
pagina = 1
while True:
  url = f"https://wodbox.es/es/43-hombre?page={pagina}"
  print(f"Extrayendo productos de la página {pagina}...")

  # Cabecera con un User-Agent que simula un navegador
  headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
  }

  # Solicitud a la página web
  response = requests.get(url, headers=headers)

  # Comprobar que la solicitud fue exitosa
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    print(f"La solicitud fue exitosa para la página {pagina}")

    # Extraer los productos individuales utilizando el 'article' con clase 'ajax_block_product'
    productos = soup.find_all("article", class_="ajax_block_product")

    if not productos:
        print(f"No se encontraron productos en la página {pagina}")
        break

    # Procesar cada producto
    for producto in productos:
        nombre = producto.find("h3", class_="s_title_block").get_text().strip()

        # Limpiar el precio: eliminar el símbolo de euro y reemplazar la coma por un punto
        precio_texto = producto.find("span", class_="price").get_text().strip()
        precio_limpio = precio_texto.replace('€', '').replace(',', '.').strip()
        # Convertir el precio a float
        try:
          precio = float(precio_limpio)
        except ValeuError:
          print(f"Error al convertir el precio: {precio_texto}")
          continue

        enlace = producto.find("h3", class_="s_title_block").find("a")["href"]

        # Insertar en la base de datos
        insertar_producto(nombre, precio, enlace)

        print(f"Producto: {nombre}, Precio: {precio}, Enlace: {enlace}")


    # Buscar el enlace a la siguiente página
    next_page = soup.find("a", class_="page-link next js-search-link")

    # Si no se encuentra la siguiente página, detener el bucle
    if not next_page:
      print("No se encontraron más páginas. Finalizando extracción")
      break

    # Incrementar el número de página
    pagina += 1
  else:
    print(f"Error en la solicitud para la página {pagina}: {response.status_code}")
    break

# Cerrar la conexión a la base de datos
cursor.close()
db_connection.close()
print("Conexión cerrada")

Extrayendo productos de la página 1...
La solicitud fue exitosa para la página 1
Datos insertados exitosamente
Producto: Zapatillas de entrenamiento UA Reign Lifter unisex, Precio: 200.0, Enlace: https://wodbox.es/es/under-armour/7222-30432-zapatillas-de-entrenamiento-ua-reign-lifter-unisex.html#/41-tallaje_calzado-425
Datos insertados exitosamente
Producto: Zapatillas de entrenamiento UA Reign Lifter unisex, Precio: 200.0, Enlace: https://wodbox.es/es/under-armour/7221-30435-zapatillas-de-entrenamiento-ua-reign-lifter-unisex.html#/23-tallaje_calzado-40
Datos insertados exitosamente
Producto: UNDER ARMOUR REIGN 6 TRAINING SHOES, Precio: 119.0, Enlace: https://wodbox.es/es/under-armour/7220-30430-under-armour-reign-6-training-shoes.html#/40-tallaje_calzado-42
Datos insertados exitosamente
Producto: UNDER ARMOUR REIGN 6 TRAINING SHOES, Precio: 119.0, Enlace: https://wodbox.es/es/under-armour/7219-30418-under-armour-reign-6-training-shoes.html#/37-tallaje_calzado-41
Datos insertados exito

# Consultas a la BBDD

In [25]:
# Función para reconectar la base de datos si es necesario
def conectar_base_datos():
    return mysql.connector.connect(
      host="bs3hiwdqhdndm8eziyc3-mysql.services.clever-cloud.com", # Host de la conexión CleverCloud-MySQLWorkbench
      user="u1j2oebciw6h6efv", # Usuario de Clever Cloud
      password="nyYM8NtWc89pfPXk7M0D", # Contraseña de Clever Cloud
      database="bs3hiwdqhdndm8eziyc3", # Nombre de la base de datos de MySQLWorkbench
      port=3306, # Puerto de la conexión CleverCloud-MySQLWorkbench
      connection_timeout=600 # 10 min de tiempo de espera
    )

In [23]:
# Función para ejecutar consultas y devolver resultados
def ejecutar_consulta(query):
    try:
      cursor.execute(query)
      resultados = cursor.fetchall()  # Obtener todos los resultados de la consulta
      return resultados
    except mysql.connector.OperationalError as e:
        print(f"Error en la conexión: {e}. Intentando reconectar...")
        # Intentamos reconectar
        db_connection = conectar_base_datos()
        cursor = db_connection.cursor()
        cursor.execute(query)
        resultados = cursor.fetchall()
        return resultados

In [26]:
# Conectar a la base de datos y crear el cursor
db_connection = conectar_base_datos()
cursor = db_connection.cursor()

In [27]:
# Consulta 1: Los 10 productos más baratos
consulta_1 = """
    SELECT nombre, precio, enlace_producto
    FROM productos_hombre
    ORDER BY precio ASC
    LIMIT 10;
"""
productos_mas_baratos = ejecutar_consulta(consulta_1)
print("Los 10 productos más baratos:")
for producto in productos_mas_baratos:
    print(producto)

Los 10 productos más baratos:
('CAMISETA RUN ESSENTIALS SPEEDWICK', Decimal('12.48'), 'https://wodbox.es/es/reebok-/4068-17278-camiseta-run-essentials-speedwick.html#/1-talla-s')
('REEBOK FULGERE', Decimal('14.00'), 'https://wodbox.es/es/reebok/5402-reebok-fulgere.html')
('CAMISETA RUN ESSENTIALS ACTIVCHILL', Decimal('14.98'), 'https://wodbox.es/es/reebok-/3860-16492-camiseta-run-essentials-activchill.html#/1-talla-s')
('MALLAS CORTAS BOSTON TRACK CLUB', Decimal('17.48'), 'https://wodbox.es/es/reebok/4135-17650-mallas-cortas-boston-track-club.html#/1-talla-s')
('REEBOK FULGERE', Decimal('17.50'), 'https://wodbox.es/es/reebok/5949-26153-reebok-fulgere.html#/38-tallaje_calzado-405')
('MALLA UNDER ARMOUR COMPRESIVA PARA HOMBRE', Decimal('17.50'), 'https://wodbox.es/es/under-armour/6563-malla-under-armour-compresiva-para-hombre.html')
('REEBOK TAPE SHORT', Decimal('17.50'), 'https://wodbox.es/es/reebok/5763-25609-reebok-tape-short.html#/2-talla-m')
('Camiseta de manga corta UA I Will', Dec

In [28]:
# Consulta 2: Los 10 productos más baratos que contienen 'zapatillas' en el nombre
consulta_2 = """
    SELECT nombre, precio, enlace_producto
    FROM productos_hombre
    WHERE nombre LIKE '%zapatillas%'
    ORDER BY precio ASC
    LIMIT 10;
"""
productos_zapatillas = ejecutar_consulta(consulta_2)
print("\nLos 10 productos más baratos que contienen 'zapatillas' en el nombre:")
for producto in productos_zapatillas:
    print(producto)


Los 10 productos más baratos que contienen 'zapatillas' en el nombre:
('Zapatillas de entrenamiento UA TriBase Reign 5', Decimal('78.00'), 'https://wodbox.es/es/under-armour/7128-30210-zapatillas-de-entrenamiento-ua-tribase-reign-5-.html#/37-tallaje_calzado-41')
('Zapatillas Froning Nano X3', Decimal('90.00'), 'https://wodbox.es/es/reebok/6874-29216-zapatillas-froning-nano-x3.html#/38-tallaje_calzado-405')
('Zapatillas Froning Nano X3', Decimal('90.00'), 'https://wodbox.es/es/reebok/6873-29190-zapatillas-froning-nano-x3.html#/19-tallaje_calzado-36')
('Zapatillas de entrenamiento UA Reign Lifter unisex', Decimal('200.00'), 'https://wodbox.es/es/under-armour/7222-30432-zapatillas-de-entrenamiento-ua-reign-lifter-unisex.html#/41-tallaje_calzado-425')
('Zapatillas de entrenamiento UA Reign Lifter unisex', Decimal('200.00'), 'https://wodbox.es/es/under-armour/7221-30435-zapatillas-de-entrenamiento-ua-reign-lifter-unisex.html#/23-tallaje_calzado-40')


In [29]:
# Consulta 3: El precio medio de todos los productos
consulta_3 = """
    SELECT AVG(precio)
    FROM productos_hombre;
"""
precio_medio = ejecutar_consulta(consulta_3)
print("\nEl precio medio de todos los productos:")
print(precio_medio[0][0])


El precio medio de todos los productos:
53.220374


In [30]:
# Cerrar la conexión a la base de datos
cursor.close()
db_connection.close()
print("Conexión cerrada")

Conexión cerrada
